## Using cabinetry to interact with published models from HEPData

This notebook is a variation on the [`cabinetry` tutorials](https://github.com/cabinetry/cabinetry-tutorials) created by [Alexander Held](https://github.com/alexander-held).

In [ ]:
import cabinetry

We customize the output from `cabinetry` via a helper function. This is optional, and the `logging` module can be used directly as well to further customize the behavior.

In [ ]:
cabinetry.set_logging()

Download a workspace from HEPData, extract it, pick a signal with `pyhf`. We use the same models that we used with `pyhf` and `funcX` from an ATLAS search for electroweakinos in final states with one lepton, missing transverse momentum and a Higgs boson decaying into two b-jets : [Eur. Phys. J. C 80 (2020) 691](https://inspirehep.net/literature/1755298). The corresponding HEPData entry is [ins1755298](https://www.hepdata.net/record/ins1755298).

In [ ]:
import json
from pathlib import Path

import pyhf
from pyhf.contrib.utils import download

In [ ]:
# locally get pyhf pallet for analysis
probability_models_url_1Lbb = (
    "https://www.hepdata.net/record/resource/1934827?view=true"
)
pallet_path = Path().cwd() / "input" / "1Lbb-pallet"

if not pallet_path.exists():
    download(probability_models_url_1Lbb, pallet_path)

with open(pallet_path / "BkgOnly.json") as read_file:
    workspace = pyhf.Workspace(json.load(read_file))

with open(pallet_path / "patchset.json") as read_file:
    patchset = pyhf.PatchSet(json.load(read_file))

workspace = patchset.apply(workspace, "C1N2_Wh_hbb_800_300")
cabinetry.workspace.save(workspace, "1Lbb.json")

The `1Lbb.json` workspace is now ready to be used. We will run a maximum likelihood fit with `cabinetry` and visualize the results. First, we have a brief look at the content of the workspace:

In [ ]:
!pyhf inspect 1Lbb.json | head -n 6

The fit model specified in the workspace is created next.

In [ ]:
workspace = cabinetry.workspace.load("1Lbb.json")
model, data = cabinetry.model_utils.model_and_data(workspace)

We can take a look at a yield table for this model. We first generate the pre-fit model prediction, and then pass it to a function to produce a yield table from it.

In [ ]:
model_prefit = cabinetry.model_utils.prediction(model)
cabinetry.tabulate.yields(model_prefit, data)

We can also visualize the pre-fit model prediction and compare it to data. the `visualize.data_mc` function returns a list of dictionaries containing the `matplotlib` figures, which we could use to customize them as needed. We do not need to customize anything here.

In [ ]:
cabinetry.visualize.data_mc(model_prefit, data);

Next up is a maximum likelihood fit:

In [ ]:
fit_results = cabinetry.fit.fit(model, data)

We can now visualize the post-fit distributions. To do so, we need a post-fit model prediction. It is obtained like the pre-fit model prediction, but this time with an additional argument to pass in the fit results.

In [ ]:
model_postfit = cabinetry.model_utils.prediction(model, fit_results=fit_results)
cabinetry.visualize.data_mc(model_postfit, data);

The nuisance parameter pulls and correlations are visualized below.

In [ ]:
cabinetry.visualize.pulls(fit_results, exclude="mu_SIG")

In [ ]:
cabinetry.visualize.correlation_matrix(fit_results, pruning_threshold=0.2)